# Комбинаторика — генерация объекта по номеру и наоборот

* Обзор комбинаторных объектов
* Генерация объекта по номеру
* Генерация номера по объекту
* Динамика по цифрам 

## Обзор комбинаторных объектов
Вы уже должны уметь генерировать в лексикографическом порядке следующие комбинаторные объекты:
* 2-ичные числа размера N (то же самое, что все подмножства N-элементного множества)
* p-ичные числа размера N
* сочетания из N по K (то же самое, что все K-элементные подмножества N-элементного множества)
* перестановки размера N


Возможно, вы умеете работать и с более сложными (или просто непривычными) объектами:

* сочетания с повторениями из N по K
* размещения из N по K
* правильные скобочные последовательности размера 2N
* разбиения числа N на K слагаемых
* разбиения числа N на слагаемые

Рассмотрим более сложный пример со всеми упорядоченными разбиениями N на слагаемые.

In [13]:
global_n = 0

def gen(n, prefix=[]):
    global global_n
    if sum(prefix) == n:
        print '{}: {}={}'.format(global_n, n, '+'.join([str(x) for x in prefix]))
        global_n += 1
        return
    if sum(prefix) > n:
        return
    for i in range(prefix[-1] if prefix else 1, n + 1):
        gen(n, prefix + [i])

gen(8)

0: 8=1+1+1+1+1+1+1+1
1: 8=1+1+1+1+1+1+2
2: 8=1+1+1+1+1+3
3: 8=1+1+1+1+2+2
4: 8=1+1+1+1+4
5: 8=1+1+1+2+3
6: 8=1+1+1+5
7: 8=1+1+2+2+2
8: 8=1+1+2+4
9: 8=1+1+3+3
10: 8=1+1+6
11: 8=1+2+2+3
12: 8=1+2+5
13: 8=1+3+4
14: 8=1+7
15: 8=2+2+2+2
16: 8=2+2+4
17: 8=2+3+3
18: 8=2+6
19: 8=3+5
20: 8=4+4
21: 8=8


# Генерация объекта по номеру

Генерировать все-все перестановки - это здорово. Еще и в лексикографическом порядке. Но давайте теперь решим более точечную задачу. А именно научимся **находить K-ю перестановку размера N, не генерируя все предыдыщие**. А именно, нам нужен алгоритм, который работает не за N!, и не за K (K может быть до N!) действий, а что-то типа O(N) действий.

Давайте сначала сгенеририруем все перестановки длины 4, чтобы на них посмотреть.

In [12]:
global_n = 0

def gen(n, prefix=[]):
    global global_n
    if len(prefix) == n:
        print '{}: {}'.format(global_n, ' '.join([str(x) for x in prefix]))
        global_n += 1
        return
    for i in range(1, n + 1):
        if i not in prefix:
            gen(n, prefix + [i])

gen(4)

0: 1 2 3 4
1: 1 2 4 3
2: 1 3 2 4
3: 1 3 4 2
4: 1 4 2 3
5: 1 4 3 2
6: 2 1 3 4
7: 2 1 4 3
8: 2 3 1 4
9: 2 3 4 1
10: 2 4 1 3
11: 2 4 3 1
12: 3 1 2 4
13: 3 1 4 2
14: 3 2 1 4
15: 3 2 4 1
16: 3 4 1 2
17: 3 4 2 1
18: 4 1 2 3
19: 4 1 3 2
20: 4 2 1 3
21: 4 2 3 1
22: 4 3 1 2
23: 4 3 2 1


Давайте подумаем, как бы мы искали **перестановку номер 16**. Если ее номер - 16, это значит, что меньше нее ровно 16 перестановок (так как нумерацию мы начинаем с нуля).

Давайте угадаем первую цифру. На 1 начинаются 3! = 6 перестановок - мало. На 2 начинаются еще 6 перестановок, суммарно 12, они все до 16-ой перестановки. А вот на 3 начинаются перестановки номер 13, 14, 15, 16, 17, 18, так что первая цифра - это **3**. На похожих рассуждениях весь алгоритм основываться и будет.

Давайте угадаем вторую цифру. На "3 1" начинаются 2! = 2 перестановки - суммарно с предыдущими 12-ю это уже 14 - все еще мало. На "3 2" начинаются еще 2 перестановки, это как раз 16 перестановок. Значит, следующая после всех этих 16-ти перестановок как и идет искомая нами перестановка номер 16. Это значит, что вторая цифра - не 2, а 4. Заметьте, что мы пропустили 3, так как она уже была! Важно их пропускать, так как в перестановке числа не повторяются. Первые две цифры - это **3 4**.

Давайте угадаем оставшиеся две цифры. Мы уже пропустили ровно 16 перестановок, поэтому видно, что это **3 4 1 2**.


### Задание
Чтобы лучше осознать, как это работает, найдите ручками перестановку длины 8 под номером 14616. Не пишите пока алгоритм.

Когда вы уже решили задание и поняли суть алгоритма, давайте углубимся в детали и напишем код:

In [8]:
factorial = [1]
for i in range(1, 20):
    factorial.append(i * factorial[-1])

def permutation_by_number(n, n_permutation, prefix=[]):
    if len(prefix) == n:
        assert n_permutation == 0
        return prefix
    for i in range(1, n + 1):
        if i not in prefix:
            if factorial[n - len(prefix) - 1] <= n_permutation:
                n_permutation -= factorial[n - len(prefix) - 1]
            else:
                return permutation_by_number(n, n_permutation, prefix + [i])

permutation_by_number(8, 14616)

[3, 8, 2, 7, 1, 4, 5, 6]

Давайте подумаем, что изменится, если заменить простые перестановки на сложные упорядоченные разбиения на слагаемые. 

Во-первых, изменится формула "сколько объектов начинаются на такой то префикс". Раньше это был простой факториал от n - len(prefix) - 1, а теперь это непростая задача - узнать, сколько разбиений на слагаемые у числа n - sum(prefix) - i, таких, что все слагаемых не менее i. Формулы для такого числа не существует, поэтому **придется посчитать отдельно двумерную динамику для этого**.

Во-вторых, изменится специфика: не нужен больше if, чтобы проверить, что новое число раньше не встречалось (это не перестановка), но нужно перебирать числа не с 1, а с предыдущего элемента(чтобы разбиение было упоряденным).

Заметьте, что из-за подсчета динамики алгоритм стал не линейным, а квадратичным, что все еще лучше, чем экспоненциальное наивное решение с генерацией всех всех разбиений.

## Генерация номера по объекту
Зная прошлую идею, не очень сложно придумать как решать и обратную задачу.
### Задание
Придумайте как и найдите ручками номер перестановки 2 3 4 1 5.

### Задание
Решите первые три задачи в этом контесте:

https://informatics.msk.ru/moodle/mod/statements/view.php?id=33206#1


## Динамика по цифрам
Похожие идеи используются, чтобы работать с числами с какими-нибудь интересными цифровыми свойствами.

Представим такую задачу: **Интересным будем называть числа, у которых все цифры слева направо нестрого убывают. Найдите, сколько интересных чисел лежат от A до B.**

Понятно, что любая задача "сколько интересных чисел от А до B" быстро сводится к задаче "сколько интересных чисел меньше равно X", потому что из ответов для двух префиксов с помощью разности можно найти ответ для отрезка.

Нам нужно посчитать какую-нибудь динамику. Например такую:

dp[n][last][is_prefix] - количество префиксов интересных чисел с такими свойствами
* префикс длины n
* последняя цифра префикса равна last
* префикс лексикографически меньше равен соответствующего префикса X
* если is_prefix == 1, то этот префикс ровно совпадает с префиксом числа X, иначе не совпадает

Обдумайте это, вся динамика строится на понимании того, что конкретно лежит в dp.

Пусть для простоты X = 4300
Заметим такие особенности:
* dp[1][0][0] = 1
* dp[1][1][0] = 1
* dp[1][2][0] = 1
* dp[1][3][0] = 1
* dp[1][4][1] = 1
* dp[1][last][is_prefix] дл остальных значений равен 0 (так как мы считаем числа, которые **меньше равны X**, и первая цифра больше 4 не может быть)
* вообще всегда dp[n][last][1] - это либо 0, либо 1 (раз is_prefix == 1, то это префикс фиксированного числа X, которого не больше одного).
* dp[2][0][0] = 5 (40, 30, 20, 10, 00)
* dp[2][1][0] = 4 (41, 31, 21, 11)
* dp[2][2][0] = 3 (42, 32, 22)
* dp[2][3][0] = 1 (33)
* dp[2][3][1] = 1 (43)
* тут можно заметить закономерность, подумать, и придумать формулу для динамики. Удобно использовать динамику вперед - то есть вот как только вы посчитали dp[n][last][0], вы тут же можете прибавить это число в еще не посчитанные dp[n+1][0][last][0], dp[n+1][1][last][0], ..., dp[n+1][last][0], так как следующая цифра в интересном числе может быть любой от 0 до last (главное чтобы нестрого убывал)
* если же вы посчитали значение dp[n][last][1], то следующей цифрой может быть не любая цифра от 0 до last, так как еще нужно соблюсти то, что все числа меньше, чем X, а значит надо перебирать до min(X[n], last):
* если last < X[n]: надо обновить dp[n+1][0][0], dp[n+1][1][0], dp[n+1][2][0], ..., dp[n+1][last][0]
* если X[n] <= last: надо обновить dp[n+1][0][0], dp[n+1][1][0], dp[n+1][2][0], ..., dp[n+1][X[n] - 1][0], dp[n+1][X[n]][**1**]


### Задание
Решите последние две задачи в этом контесте:

https://informatics.msk.ru/moodle/mod/statements/view3.php?id=33206&chapterid=193#1
